In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from biot_savart_v4_3 import parse_coil, plot_coil, slice_coil, plot_coil2
from tqdm.notebook import trange, tqdm
from helpers import get_arc_point, draw_arc, rotate, scale, rotate_point, translate

In [ ]:
# Track width and spacing
TRACK_WIDTH = 0.127
TRACK_SPACING = 0.127

# via defaults
VIA_DIAM = 0.8
VIA_DRILL = 0.4

# this is for a 1.27mm pitch pin
PIN_DIAM = 1.0
PIN_DRILL = 0.65

# this is for the PCB connector - see https://www.farnell.com/datasheets/2003059.pdf
PAD_WIDTH = 3
PAD_HEIGHT = 2
PAD_PITCH = 2.5

STATOR_HOLE_RADIUS = 5.5
HOLE_SPACING = 0.25

# PCB Edge size
STATOR_RADIUS = 30

# where to puth the mounting pins
SCREW_HOLE_DRILL_DIAM = 2.3  # 2.3mm drill for a 2mm screw
SCREW_HOLE_RADIUS = STATOR_RADIUS

# Coil params
TURNS = 16
COIL_CENTER_RADIUS = 19.95
COIL_VIA_RADIUS = 20.95


def get_points(spacing, inner_radius, outer_radius, start_angle, end_angle):
    # first calculate the angle step size from the spacing and the inner_radius
    spacing_angle = np.rad2deg(np.arctan2(spacing, inner_radius))
    print(spacing_angle)
    # now calculate the points be iterating from start_angle to end_angle with the spacing_angle
    points = []
    for angle in np.arange(start_angle, end_angle, spacing_angle * 2):
        p1 = get_arc_point(angle, inner_radius)
        points.append([p1[0], p1[1], 0, 0.5])
        p2 = get_arc_point(angle + spacing_angle, outer_radius)
        points.append([p2[0], p2[1], 0, 0.5])
        points.append([p2[0], p2[1], -0.8, 0.5])
        points.append([p1[0], p1[1], -0.8, 0.5])
    return points


coil1 = get_points(
    TRACK_SPACING + TRACK_WIDTH,
    (COIL_CENTER_RADIUS - 10),
    COIL_CENTER_RADIUS + 10,
    -15,
    15,
)
# move all the points in by COIL_CENTER_RADIUS
for p in coil1:
    p[0] -= COIL_CENTER_RADIUS
# rotate the points by 90 degrees
for p in coil1:
    p[0], p[1] = rotate_point(p[0], p[1], 90)
coil1 = np.array(coil1).T
plot_coil2(coil1)
coil1 = slice_coil(coil1, 0.1)
coil1 = coil1.T
print(coil1.shape)
print(len(coil1))

# Simple Simulation of a dipole magnet

In [ ]:
# magnetic field at a point x,y,z of a dipole magnet with moment m in the z direction
def B(x, y, z, m=0.185):
    mu0 = 4 * np.pi * 1e-7
    r = np.sqrt(x**2 + y**2 + z**2)
    return (
        np.array(
            [3 * x * z / r**5, 3 * y * z / r**5, (3 * z**2 / r**5 - 1 / r**3)]
        )
        * m
        * mu0
    )


def plot_field_slice(x, y, bx, by, mag, name="magnetic_field.png"):
    # plot the magnetic field
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.streamplot(
        x,
        y,
        bx,
        by,
        linewidth=1,
        cmap=plt.cm.inferno,
        density=2,
        arrowstyle="->",
        arrowsize=1.5,
    )

    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")
    ax.set_xlim(-0.1, 0.1)
    ax.set_ylim(-0.1, 0.1)
    ax.set_aspect("equal")

    # plot the magniture of the field as an image
    im = ax.imshow(
        mag, extent=[-0.1, 0.1, -0.1, 0.1], origin="lower", cmap=plt.cm.inferno
    )

    fig.show()
    # save the figure
    fig.savefig(name)


# # calculate the magnetic field at y = 0, over z = -1, 1 and x = -1, 1
x = np.linspace(-0.1, 0.1, 100)
z = np.linspace(-0.1, 0.1, 100)
X, Z = np.meshgrid(x, z)
Bx, By, Bz = B(X, 0, Z)

plot_field_slice(
    X,
    Z,
    Bx,
    Bz,
    np.log(np.sqrt(Bx**2 + By**2 + Bz**2)),
    "magnetic_field_side.png",
)

# # calculate the magnetic field at z = 1, over y = -1, 1 and x = -1, 1
x = np.linspace(-0.1, 0.1, 100)
y = np.linspace(-0.1, 0.1, 100)
X, Y = np.meshgrid(x, y)
Bx, By, Bz = B(X, Y, 0.01)

plot_field_slice(
    X, Y, Bx, By, np.sqrt(Bx**2 + By**2 + Bz**2), "magnetic_field_bottom.png"
)

# calculate the magnetic field in a 3d volume
x = np.linspace(-0.1, 0.1, 100)
y = np.linspace(-0.1, 0.1, 100)
z = np.linspace(-0.1, 0.1, 100)
X, Y, Z = np.meshgrid(x, y, z)
Bx, By, Bz = B(X, Y, Z)

In [ ]:
# calculate the force on a wire of length l carrying current I at a point x,y,z with a direction vector d
def F(p, d, I, l):
    return I * l * np.cross(d, B(p[0], p[1], p[2]))


def calculate_forces_on_wire_points(points):
    Fx = []
    Fy = []
    Fz = []
    # calculate the force on each point
    for i in range(len(points)):
        # calculate the direction vector
        dx = points[i][0] - points[(i + 1) % len(points)][0]
        dy = points[i][1] - points[(i + 1) % len(points)][1]
        dz = points[i][2] - points[(i + 1) % len(points)][2]
        d = np.array([dx, dy, dz])
        # get the length of d
        l = np.sqrt(dx**2 + dy**2 + dz**2)
        if l > 0:
            # normalise d
            d = d / l
            # calculate the force
            fx, fy, fz = F(points[i], d, points[i][3], l)
            Fx.append(fx)
            Fy.append(fy)
            Fz.append(fz)
        else:
            Fx.append(0)
            Fy.append(0)
            Fz.append(0)
    return Fx, Fy, Fz


# locate the loop of wire directly below the magnet
x = 0.01
y = 0
z = -0.002
r1 = 0.001
r2 = 0.01


points = coil1.copy()
# scale the points from mm to m
# shift the coil to the correct position
for i in range(len(points)):
    points[i][0] = points[i][0] / 1000 + x
    points[i][1] = points[i][1] / 1000 + y
    points[i][2] = points[i][2] / 1000 + z


# plot the points in 2D x,y
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot([p[0] for p in points], [p[1] for p in points])
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(-0.01 + x, 0.01 + x)
ax.set_ylim(-0.01 + y, 0.01 + y)
ax.set_aspect("equal")
fig.show()

Fx, Fy, Fz = calculate_forces_on_wire_points(points)


# plot the wire along with arrows showing the force
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.plot([p[0] for p in points], [p[1] for p in points], [p[2] for p in points])

print(sum(Fx) / 9.8)

# subsample the points and force vectors to make the plot clearer
points = points[::50]
Fx = Fx[::50]
Fy = Fy[::50]
Fz = Fz[::50]

ax.quiver(
    [p[0] for p in points],
    [p[1] for p in points],
    [p[2] for p in points],
    np.sqrt(Fx),
    0,
    0,
    length=0.01,
    normalize=True,
)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")
ax.set_xlim(x - 0.02, x + 0.02)
ax.set_ylim(y - 0.02, y + 0.02)
ax.set_zlim(z - 0.02, z + 0.02)
ax.set_aspect("equal")

# change the figure size
fig.set_size_inches(10, 10)

fig.show()
# coil2 = -0.01311082557350831
# coil1 =  0.0088435517657609

In [ ]:
def sweep_coil(coil, X):
    Y = np.zeros(len(X))
    Z = -0.01 * np.ones(len(X))

    # loop through the locations and calculate the forces, sum up the force in the X direction for each location
    Fx = []
    Fy = []
    Fz = []
    for p in trange(len(X)):
        points = coil.copy()
        # scale the points from mm to m
        # shift the coil to the correct position
        for i in range(len(points)):
            points[i][0] = points[i][0] / 1000 + X[p]
            points[i][1] = points[i][1] / 1000 + Y[p]
            points[i][2] = points[i][2] / 1000 + Z[p]
        Fx_, Fy_, Fz_ = calculate_forces_on_wire_points(points)
        Fx.append(sum(Fx_))
        Fy.append(sum(Fy_))
        Fz.append(sum(Fz_))
    return Fx, Fy, Fz


# sweep the coild from -3cm to 3cm in 0.01m steps
X = np.linspace(-0.03, 0.03, 100)
Fx_1_straight, Fy_1_straight, Fz_1_straight = sweep_coil(coil1, X)

In [ ]:
# plot the force as a function of x
plt.plot(X, -np.array(Fx_1_straight) / 9.8, label="coil1", color="red")
# plot a dotted line along y = 0
plt.plot([X[0], X[-1]], [0, 0], "--")

In [ ]:
# plot the force as a function of x
plt.plot(X, -np.array(Fz_1_straight) / 9.8, label="coil1", color="red")
# plot a dotted line along y = 0
plt.plot([X[0], X[-1]], [0, 0], "--")

In [ ]:
# instead of sweeping horizontally, we'll sweep the coils around a circle
def sweep_coil_cirlc(coil, coil_center_radius, theta):
    X = coil_center_radius * np.cos(np.deg2rad(theta))
    Y = coil_center_radius * np.sin(np.deg2rad(theta))
    Z = -0.01 * np.ones(100)

    # loop through the locations and calculate the forces, sum up the force in the X direction for each location
    Torque = []
    Fx = []
    Fy = []
    Fz = []
    for p in trange(len(theta)):
        angle = np.deg2rad(theta[p]) - np.pi / 2
        x = X[p]
        y = Y[p]
        z = Z[p]

        points = coil.copy()
        for i in range(len(points)):
            px = points[i][0] / 1000
            py = points[i][1] / 1000
            pz = points[i][2] / 1000
            # rotate the points so the coil is correctly oriented
            points[i][0] = px * np.cos(angle) - py * np.sin(angle) + x
            points[i][1] = (
                px * np.sin(angle) + py * np.cos(angle) + y - coil_center_radius
            )
            points[i][2] = pz + z
        # feel the force
        Fx_, Fy_, Fz_ = calculate_forces_on_wire_points(points)
        Fx.append(sum(Fx_))
        Fy.append(sum(Fy_))
        Fz.append(sum(Fz_))
        # calculate the torque - which should be 90 degress to the angle
        torque_angle = np.deg2rad(theta[p] - 90)
        Torque.append(sum(Fx_) * np.cos(torque_angle) + sum(Fy_) * np.sin(torque_angle))

    return Fx, Fy, Fz, Torque


# sweep the coils from -45 to 45 degrees in 1 degree steps
theta = np.linspace(0, 180, 100)
Fx_1_curve, Fy_1_curve, Fz_1_curve, Torque_1 = sweep_coil_cirlc(
    coil1, 19.5 / 1000, theta
)

In [ ]:
plt.plot(theta, -np.array(Torque_1) / 9.8, label="coil1", color="red")
# plot a dotted line along y = 0
plt.plot([theta[0], theta[-1]], [0, 0], "--")

In [ ]:
# plot arrows for the Fx and Fy components
X = 19.5 * np.cos(np.deg2rad(theta)) / 1000
Y = 19.5 * np.sin(np.deg2rad(theta)) / 1000
plt.quiver(X[::5], Y[::5], Fx_1_curve[::5], Fy_1_curve[::5], color="red")
# make the axis equal so the arrows are not stretched
plt.axis("equal")

In [ ]:
plt.plot(theta, -np.array(Fz_1_curve) / 9.8, label="coil1", color="red")
plt.plot(theta, np.array(Fz_2_curve) / 9.8, label="coil2", color="blue")
# plot a dotted line along y = 0
plt.plot([theta[0], theta[-1]], [0, 0], "--")

In [ ]:
# instead of sweeping horizontally, we'll sweep the coils around a circle
def sweep_coil_cirlc(coil, coil_center_radius):
    # sweep the coils from -45 to 45 degrees in 1 degree steps
    theta = np.linspace(0, 180, 20)
    X = coil_center_radius * np.cos(np.deg2rad(theta))
    Y = coil_center_radius * np.sin(np.deg2rad(theta))
    Z = 1 * np.ones(100)

    # loop through the locations and calculate the forces, sum up the force in the X direction for each location
    Fx = []
    Fy = []
    Fz = []
    for p in trange(len(theta)):
        angle = np.deg2rad(theta[p]) - np.pi / 2
        x = X[p]
        y = Y[p]
        z = Z[p]

        points = coil.copy()
        for i in range(len(points)):
            px = points[i][0] / 1000
            py = points[i][1] / 1000
            pz = points[i][2] / 1000
            # rotate the points so the coil is correctly oriented
            points[i][0] = px * np.cos(angle) - py * np.sin(angle) + x
            points[i][1] = (
                px * np.sin(angle) + py * np.cos(angle) + y - coil_center_radius
            )
            points[i][2] = pz + z
        plt.plot([p[0] for p in points], [p[1] for p in points], linewidth=0.5)
        # add the torque arrow to the plot
        torque_angle = np.deg2rad(theta[p] - 90)
        torque_x1 = x
        torque_y1 = y
        torque_x2 = x + 0.01 * np.cos(torque_angle)
        torque_y2 = y + 0.01 * np.sin(torque_angle)
        plt.arrow(
            torque_x1,
            torque_y1,
            torque_x2 - torque_x1,
            torque_y2 - torque_y1,
            head_width=0.001,
            head_length=0.002,
            fc="k",
            ec="k",
        )


sweep_coil_cirlc(coil2, 19.5 / 1000)